<a href="https://colab.research.google.com/github/fjcorreiaa/etl_python_DIO/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
import pandas as pd
import requests
import json

In [2]:
# URL Endpoint
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [3]:
# df = pd.read_csv('users.csv')
# user_ids = df['UserID'].tolist()

ids = [1035, 1039, 1040, 1041, 1042]
df = pd.DataFrame(ids, columns = ['UserID'])
user_ids = ids

In [4]:
def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None


## **T**ransform

Utilizando o limite de crédito do usuário para gerar uma news

In [5]:
# Função para obter o limite de um usuário
def get_limit(user):
    account = user.get('account')
    if user:
      user_limit = account.get("limit")
      return user_limit
    else:
      print(f'Falha ao obter limite do usuário {user.get("id")}')
      return None;


In [6]:
#Get User
users = [get_user(id) for id in user_ids]

#Get Limit
limits = [get_limit(user) for user in users]

df['limit_value'] = limits

# Exibir o DataFrame resultante
print(df)

   UserID  limit_value
0    1035        700.0
1    1039        100.0
2    1040        100.0
3    1041      50000.0
4    1042        500.0


In [11]:
def generate_ai_news(user):
    limit_value = user.get('account', {}).get('limit', 0)

    if limit_value < 300:
        mensagem = f"Olá {user.get('name')}! Precisando de mais limite? Temos uma solução imperdível para você."
    else:
        mensagem = f"Olá {user.get('name')}! Aproveite o cashback em nossa loja virtual."

    return mensagem

In [13]:
for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Olá Luciana! Aproveite o cashback em nossa loja virtual.
Olá Julio! Precisando de mais limite? Temos uma solução imperdível para você.
Olá Ana! Precisando de mais limite? Temos uma solução imperdível para você.
Olá Valter! Aproveite o cashback em nossa loja virtual.
Olá Garfield! Aproveite o cashback em nossa loja virtual.


## **Load**

In [14]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False


In [15]:
for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Luciana updated? True!
User Julio updated? True!
User Ana updated? True!
User Valter updated? True!
User Garfield updated? True!
